In [ ]:
import pandas as pd
df=pd.read_csv("d:/data/customer/customer.csv")
df.head()

In [ ]:
df["Churn"].value_counts()
#유지 2850, 이탈 483 불균형 데이터셋

In [ ]:
#변수 정의
train_cols=df.columns[0:16]
X=df[train_cols] #독립변수
y=df["Churn"] #종속변수

In [ ]:
#언더샘플링
from imblearn.under_sampling import RandomUnderSampler
#적은 데이터셋을 기준으로 랜덤샘플링
X_sample,y_sample=RandomUnderSampler(random_state=0).fit_sample(X,y)
#데이터프레임으로 변환
X_samp=pd.DataFrame(data=X_sample,columns=train_cols)
y_samp=pd.DataFrame(data=y_sample,columns=["Churn"])
df_samp=pd.concat([X_samp,y_samp],axis=1)

In [ ]:
df_samp["Churn"].value_counts() 
# 483:483으로 언더샘플링

In [ ]:
X=df_samp[train_cols] #독립변수
y=df_samp["Churn"] #종속변수

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
#데이터셋을 8:2로 구분
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=10)
#트리 모형 생성
model=DecisionTreeClassifier(random_state=0)
#모형 훈련
model.fit(X_train,y_train)

In [ ]:
#예측정확도 확인
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
#언더샘플링(과적합)
# 1.0
# 0.845360824742268

In [ ]:
#트리 모형 생성(max_depth를 3으로 가지치기)
model=DecisionTreeClassifier(max_depth=3,random_state=0)
#모형 훈련
model.fit(X_train,y_train)

In [ ]:
#예측정확도 확인
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))
#언더샘플링
# 0.8354922279792746
# 0.8556701030927835

In [ ]:
from sklearn.tree import export_graphviz
import pydotplus 
import graphviz
from IPython.display import Image

dot_data=export_graphviz(model,out_file=None,
    feature_names=train_cols, class_names=["0","1"],
    filled=True, rounded=True, special_characters=True)
graph=pydotplus.graph_from_dot_data(dot_data)    
Image(graph.create_png())

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
font_name=font_manager.FontProperties(fname="c:/windows/fonts/malgun.ttf").get_name()
rc("font",family=font_name)

def plot_tree(model):
    n_features=X.shape[1] #독립변수의 갯수
    plt.rcParams["figure.figsize"]=(16,9) #그래프의 가로,세로 인치
    plt.rcParams["font.size"]=17 #그래프의 폰트 사이즈
    plt.barh(range(n_features),model.feature_importances_,
             align="center")
    plt.yticks(np.arange(n_features),train_cols) 
    plt.xlabel("특성 중요도")
    plt.ylabel("특성")
    plt.ylim(-1,n_features)
    
plot_tree(model)     #특성 중요도 그래프 출력
# 언더샘플링 : Day Charge > CustServ Calls > Int'l Plan

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#랜덤포레스트 모델 생성
#n_estimators : 트리갯수
forest=RandomForestClassifier(
    n_estimators=100,random_state=100,max_depth=4)
forest.fit(X_train, y_train)  #모델 훈련

In [ ]:
print(forest.score(X_train,y_train)) #학습용 데이터셋의 정확도
print(forest.score(X_test,y_test)) #검증용 데이터셋의 정확도
#언더샘플링
# 0.8989637305699482
# 0.8814432989690721

In [ ]:
plot_tree(forest) #트리보다 좀더 다양한 시각으로 데이터를 분석
#언더샘플링
#CustServ Calls > Day Charge > Day Mins